# Make Tables for Paper

In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from zedstat.textable import textable

In [2]:
DATA_DIR = 'results/enet_who_comparison/'
OUT_DIR = 'tables/'
FILES = ['north_h1n1', 'south_h1n1', 'north_h3n2', 'south_h3n2']

NORTH_YEARS = []
for i in np.arange(2, 23):
    YEAR = ''
    if i < 10:
        YEAR += '200' + str(i)
    else:
        YEAR += ('20' + str(i))
    if i + 1 < 10:
        YEAR += '-0' + str(i + 1)
    else:
        YEAR += '-' + str(i + 1)
    NORTH_YEARS.append(YEAR)
        
SOUTH_YEARS = []
for i in np.arange(3, 24):
    if i < 10:
        SOUTH_YEARS.append('200' + str(i))
    else:
        SOUTH_YEARS.append('20' + str(i))

## Enet vs. WHO Recommendations 

In [3]:
for FILE in FILES:
    df = pd.read_csv(DATA_DIR + FILE + '.csv')
    df = df[['name_who','name_0','name_1','ha_who_error','best_ha_enet_error','na_who_error','best_na_enet_error']]
    if FILE[:5] == 'north':
        df['Season'] = NORTH_YEARS
    else:
        df['Season'] = SOUTH_YEARS
    df = df.set_index('Season')
    df = df.rename(columns={'name_who':'WHO Recommendation',
                            'name_0':'Emergenet Recommendation 1',
                            'name_1':'Emergenet Recommendation 2',
                            'ha_who_error':'Avg. HA WHO Error',
                            'best_ha_enet_error':'Avg. HA Enet Error',
                            'na_who_error':'Avg. NA WHO Error',
                            'best_na_enet_error':'Avg. NA Enet Error'})
    # textable(df, tabname = OUT_DIR + FILE + '.tex', FORMAT='%1.1f')

## Enet Improvement Table

In [3]:
subtype = 2*(['H1N1']*3 + ['H3N2']*3)
gene = 6*['HA'] + 6*['NA']
hemisphere = 4*['North', 'South', 'Average']

who_err_20 = [] # 2 decade error
enet_err_20 = [] 
who_err_10 = [] # 1 decade error
enet_err_10 = [] 
who_err_15_19 = [] # 2015-2019 error
enet_err_15_19 = [] 

for SEGMENT in ['ha', 'na']:
    for i in range(0, 4, 2):
        north_df = pd.read_csv(DATA_DIR + FILES[i] + '.csv')[:-1]
        south_df = pd.read_csv(DATA_DIR + FILES[i + 1] + '.csv')[:-1]
        WHO = SEGMENT + '_who_error'
        ENET = 'best_' + SEGMENT + '_enet_error'
        # WHO north
        who_err_20.append(np.mean(north_df[WHO]))
        who_err_10.append(np.mean(north_df[WHO][-10:]))
        who_err_15_19.append(np.mean(north_df[WHO][12:17]))
        # Enet north
        enet_err_20.append(np.mean(north_df[ENET]))
        enet_err_10.append(np.mean(north_df[ENET][-10:]))
        enet_err_15_19.append(np.mean(north_df[ENET][12:17]))
        # WHO south
        who_err_20.append(np.mean(south_df[WHO]))
        who_err_10.append(np.mean(south_df[WHO][-10:]))
        who_err_15_19.append(np.mean(south_df[WHO][12:17]))
        # Enet south
        enet_err_20.append(np.mean(south_df[ENET]))
        enet_err_10.append(np.mean(south_df[ENET][-10:]))
        enet_err_15_19.append(np.mean(south_df[ENET][12:17]))
        # WHO average
        who_err_20.append(np.mean(who_err_20[-2:]))
        who_err_10.append(np.mean(who_err_10[-2:]))
        who_err_15_19.append(np.mean(who_err_15_19[-2:]))
        # Enet average
        enet_err_20.append(np.mean(enet_err_20[-2:]))
        enet_err_10.append(np.mean(enet_err_10[-2:]))
        enet_err_15_19.append(np.mean(enet_err_15_19[-2:]))

improvement_20 = 100 * (np.array(who_err_20) - np.array(enet_err_20)) / np.array(who_err_20)
improvement_10 = 100 * (np.array(who_err_10) - np.array(enet_err_10)) / np.array(who_err_10)
improvement_15_19 = 100 * (np.array(who_err_15_19) - np.array(enet_err_15_19)) / np.array(who_err_15_19)

enet_improvement = pd.DataFrame({'Subtype':subtype,
                                 'Gene':gene,
                                 'Hemisphere':hemisphere,
                                 'WHO Error1':who_err_20,
                                 'Enet Error1':enet_err_20,
                                 'Improvement1 (%)':improvement_20,
                                 'WHO Error2':who_err_10,
                                 'Enet Error2':enet_err_10,
                                 'Improvement2 (%)':improvement_10,
                                 'WHO Error3':who_err_15_19,
                                 'Enet Error3':enet_err_15_19,
                                 'Improvement3 (%)':improvement_15_19})
enet_improvement.to_csv(DATA_DIR + 'enet_improvement.csv', index=False)

In [10]:
enet_improvement = pd.read_csv(DATA_DIR + 'enet_improvement.csv', index_col=0)
# textable(enet_improvement, tabname = OUT_DIR + 'enet_improvement.tex', FORMAT='%1.2f')
enet_improvement

,Gene,Hemisphere,WHO Error1,Enet Error1,Improvement1 (%),WHO Error2,Enet Error2,Improvement2 (%),WHO Error3,Enet Error3,Improvement3 (%)
Subtype,,,,,,,,,,,
H1N1,HA,North,14.7250,4.1450,71.850594,10.840,4.040,62.730627,11.48,2.88,74.912892
H1N1,HA,South,15.0250,10.1150,32.678869,9.770,4.240,56.601842,9.80,4.40,55.102041
H1N1,HA,Average,14.8750,7.1300,52.067227,10.305,4.140,59.825328,10.64,3.64,65.789474
H3N2,HA,North,10.0400,6.2700,37.549801,8.730,6.990,19.931271,6.40,4.20,34.375000
H3N2,HA,South,8.9750,4.6850,47.799443,8.240,4.040,50.970874,6.98,3.58,48.710602
H3N2,HA,Average,9.5075,5.4775,42.387589,8.485,5.515,35.002946,6.69,3.89,41.853513
H1N1,NaN,North,10.2000,3.6000,64.705882,7.180,4.380,38.997214,9.96,3.76,62.248996
H1N1,NaN,South,11.3450,7.9450,29.969149,6.960,4.060,41.666667,9.00,4.80,46.666667
H1N1,NaN,Average,10.7725,5.7725,46.414481,7.070,4.220,40.311174,9.48,4.28,54.852321
